In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from datetime import datetime, timedelta
import orekit
import time
import pymsis
from pymsis import msis
import matplotlib.dates as mdates
from scipy.optimize import curve_fit
import hrd_20250608.utilities_ds as u
from orekit.pyhelpers import download_orekit_data_curdir, setup_orekit_curdir
from os import path
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from scipy.linalg import expm, logm
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error
import hrd_20250608.rope_class_hrd as rope 
import glob

In [4]:

columns = [
    'date', 'time', 'GPS', 'alt', 'lon', 'lat', 'lst', 'arglat',
    'accelerometer_density', 'dens_mean', 'flag_dens', 'flag_dens_mean'
]

file_paths = sorted(glob.glob('hrd_20250608/champ_data/*.txt'))

df_list = []

for file_path in file_paths:
    df = pd.read_csv(
        file_path,
        delim_whitespace=True,
        comment='#',
        names=columns,
        on_bad_lines='skip'  # Skip lines that cannot be parsed
    )
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.set_index('datetime', inplace=True)
    df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
    df_list.append(df)

champ_all = pd.concat(df_list)

champ_all.sort_index(inplace=True)


columns = [
    'date', 'time', 'GPS', 'alt', 'lon', 'lat', 'lst', 'arglat',
    'accelerometer_density', 'dens_mean', 'flag_dens', 'flag_dens_mean'
]

file_paths = sorted(glob.glob('hrd_20250608/grace_data/*.txt'))

df_list = []

for file_path in file_paths:
    df = pd.read_csv(
        file_path,
        delim_whitespace=True,
        comment='#',
        names=columns
    )
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.set_index('datetime', inplace=True)
    df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
    df_list.append(df)

grace_all = pd.concat(df_list)

grace_all.sort_index(inplace=True)

columns = [
    'date', 'time', 'GPS', 'alt', 'lon', 'lat', 'lst', 'arglat',
    'accelerometer_density'
]

file_paths = sorted(glob.glob('hrd_20250608/swarm_data/SA_*.txt'))

df_list = []

for file_path in file_paths:
    df = pd.read_csv(
        file_path,
        delim_whitespace=True,
        comment='#',
        names=columns
    )
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.set_index('datetime', inplace=True)
    df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
    df_list.append(df)

swarma_all = pd.concat(df_list)

swarma_all.sort_index(inplace=True)

file_paths = sorted(glob.glob('hrd_20250608/swarm_data/SC_*.txt'))

df_list = []

for file_path in file_paths:
    df = pd.read_csv(
        file_path,
        delim_whitespace=True,
        comment='#',
        names=columns
    )
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.set_index('datetime', inplace=True)
    df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
    df_list.append(df)

swarmc_all = pd.concat(df_list)

swarmc_all.sort_index(inplace=True)

In [5]:
champ_all

,alt,lon,lat,lst,arglat,accelerometer_density,dens_mean,flag_dens,flag_dens_mean
datetime,,,,,,,,,
2001-05-04 12:00:00,437482.545,-7.169527,-62.689635,11.572,297.328171,2.932771e-12,NaN,NaN,NaN
2001-05-04 12:00:30,437314.192,-6.887576,-60.769854,11.600,299.264319,3.028263e-12,NaN,NaN,NaN
2001-05-04 12:01:00,437137.751,-6.651596,-58.848457,11.624,301.200295,3.093940e-12,NaN,NaN,NaN
2001-05-04 12:01:30,436955.212,-6.453548,-56.925614,11.645,303.136094,3.219620e-12,NaN,NaN,NaN
2001-05-04 12:02:00,436768.634,-6.287146,-55.001455,11.665,305.071707,3.295522e-12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2010-09-04 20:03:30,256257.758,16.658131,73.001783,21.182,106.918428,2.158161e-11,NaN,NaN,NaN
2010-09-04 20:04:00,256001.089,17.554233,71.020416,21.250,108.935221,2.187903e-11,NaN,NaN,NaN
2010-09-04 20:04:30,255720.956,18.263595,69.033973,21.306,110.951973,2.102908e-11,NaN,NaN,NaN


In [6]:
grace_all

,alt,lon,lat,lst,arglat,accelerometer_density,dens_mean,flag_dens,flag_dens_mean
datetime,,,,,,,,,
2002-04-02 00:00:00,516094.784,161.091557,-51.734127,10.739,308.418238,1.495161e-12,NaN,NaN,NaN
2002-04-02 00:01:30,515068.086,160.938410,-46.043609,10.754,314.115342,1.566026e-12,NaN,NaN,NaN
2002-04-02 00:02:00,514701.226,160.877322,-44.145342,10.758,316.014165,1.596309e-12,NaN,NaN,NaN
2002-04-02 00:04:30,512789.207,160.520734,-34.643310,10.776,325.506933,1.715442e-12,NaN,NaN,NaN
2002-04-02 00:05:00,512407.745,160.441728,-32.740817,10.779,327.405254,1.741611e-12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2009-11-29 22:54:00,458430.478,107.753559,38.085597,6.084,142.028719,9.816559e-14,NaN,NaN,NaN
2009-11-29 22:55:00,457332.890,107.603983,34.229395,6.090,145.878419,8.737271e-14,NaN,NaN,NaN
2009-11-29 22:55:30,456829.933,107.525770,32.300114,6.093,147.803276,9.934695e-14,NaN,NaN,NaN


In [7]:
swarma_all

,alt,lon,lat,lst,arglat,accelerometer_density
datetime,,,,,,
2013-11-28 00:00:00,518180.489,-152.381468,-49.294556,14.044,310.808740,6.489984e-13
2013-11-28 00:00:30,517273.110,-152.324478,-47.401812,14.056,312.706744,6.546779e-13
2013-11-28 00:01:00,516352.776,-152.280063,-45.507821,14.067,314.604952,6.603022e-13
2013-11-28 00:01:30,515422.463,-152.246633,-43.612608,14.078,316.503368,6.657121e-13
2013-11-28 00:02:00,514485.169,-152.222840,-41.716197,14.087,318.401998,6.709199e-13
...,...,...,...,...,...,...
2024-05-31 23:59:20,495053.318,74.188291,-61.787368,4.970,298.132029,3.981371e-13
2024-05-31 23:59:30,494838.136,74.274808,-61.156186,4.979,298.768338,4.526104e-13
2024-05-31 23:59:30,494838.136,74.274808,-61.156186,4.979,298.768338,4.168664e-13


In [33]:
def build_meas_data(start_date, end_date, msis_version, dataset):
    
    subset = dataset.loc[start_date:end_date]
    
    timestamps = subset.index.values
    latitude_values = subset.lat.values # degrees
    local_time_values = subset.lst.values # hours
    longitude_values = subset.lon.values
    altitude_values = subset.alt.values/1000. # km
    lla_array = np.vstack((latitude_values, local_time_values, altitude_values)).T.reshape((-1, 3))

    result = msis.calculate(
        timestamps,
        longitude_values,
        latitude_values,
        altitude_values, 
        geomagnetic_activity=-1,
        version = msis_version
    )

    msis_rho = result[:, 0]
    accelerometer_density = subset["accelerometer_density"].values

    meas_df = pd.DataFrame({
        'timestamp': timestamps,
        'accelerometer_density': accelerometer_density,
        'msis_rho': msis_rho,
        'latitude': latitude_values,
        'local_time': local_time_values,
        'longitude': longitude_values,
        'altitude': altitude_values
    })

    return meas_df


# def run_demo(start_date, end_date, rope_density, msis_version, dataset, bias_propagation):

    # timestamps, accelerometer_density, ensemble_density, dmd_density, \
    #     density_std, msis_rho, outputs_df, latitudes, lst_values = \
    #         build_output_data(start_date, end_date, rope_density, msis_version, dataset)
    
    # debias_dataset = bias_calculator(start_date, bias_propagation, dataset)
    
    # debias_ratio_df = debias_dataset.groupby(['hour']).agg(mean_ratio = ('debias_ratio', 'mean')).reset_index().copy()
    # debias_density_df = pd.DataFrame({'datetime': timestamps,
    # 'hour': pd.to_datetime(timestamps).hour,
    #     'hour_minute': pd.to_datetime(timestamps).hour + pd.to_datetime(timestamps).minute/60., 
    #         'hms': pd.to_datetime(timestamps).second + pd.to_datetime(timestamps).minute*100 + pd.to_datetime(timestamps).hour*1000,
    #             'den': ensemble_density})
    # debias_density_df['t1'] = np.cos(np.pi*2.*debias_density_df['hms'].values/omega)
    # debias_density_df['t2'] = np.sin(np.pi*2.*debias_density_df['hms'].values/omega)

    # debias_density_df = pd.merge(debias_density_df, debias_ratio_df, on=['hour'], how = 'left')
    # debias_density_df['debiased_density'] = debias_density_df['den'] * debias_density_df['mean_ratio']
    # outputs_df['debiased_ensemble_density'] = debias_density_df.debiased_density.values   

    # return outputs_df

In [34]:
msis_version = 2.1
omega = 45

# User inputs
forward_propagation_mins = 1  # forward propagation time (mins)
init_date = pd.to_datetime('2003-05-10 00:00:00')
t_increment_mins = 61

latitude_values = [89.]  # degrees
local_time_values = [8.]  # hours
altitude_values = [358.]  # km
lla_array = np.vstack((latitude_values, local_time_values, altitude_values)).T.reshape((-1, 3))

end_date = init_date + pd.Timedelta(minutes=forward_propagation_mins * (t_increment_mins - 1))


meas_df = build_meas_data(init_date, end_date, msis_version, grace_all)

# Save meas_df to an Excel sheet
excel_file_path = 'measurement_data.xlsx'
meas_df.to_excel(excel_file_path, index=False)
